<a href="https://colab.research.google.com/github/Yushamsi/Data-Ingestion-Combined-Flights/blob/main/Data-Ingestion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Ingestion

In [ ]:
import pandas as pd
import dask.dataframe as dd
import modin.pandas as mpd
import time

In [ ]:
start_time = time.time()
df_pandas = pd.read_csv('your_large_file.csv')

In [ ]:
print(f"Pandas read time: {time.time() - start_time} seconds")

In [ ]:

start_time = time.time()
df_dask = dd.read_csv('your_large_file.csv')
df_dask = df_dask.compute()  # Trigger computation to load data into memory

In [ ]:
print(f"Dask read time: {time.time() - start_time} seconds")

In [ ]:
start_time = time.time()
df_modin = mpd.read_csv('your_large_file.csv')

In [ ]:
print(f"Modin read time: {time.time() - start_time} seconds")